In [92]:
import random

# ================================================
# Implementación clase abstracta algoritmo genético 
# ================================================
# Definir clase abstracta Problema_Genetico 
# Propiedades:
# - genes: lista de genes usados en el genotipo de los estados.
# - longitud_individuos: longitud de los cromosomas
# Métodos:
# - decodifica: función de obtiene el fenotipo a partir del genotipo.
# - fitness: función de valoración.
# - muta: mutación de un cromosoma 
# - cruza: cruce de un par de cromosomas

# En la definición de clase no se especifica si el problema es
# de maximización o de minimización, esto se hace con el
# parámetro en el algoritmo genético que se implemente.

class Problema_Genetico(object):
    # Constructor
    def __init__(self, genes, fun_decodificar, fun_cruzar, fun_mutar, fun_fitness, longitud_individuo):
        self.genes = genes
        self.fun_decodificar = fun_decodificar
        self.fun_cruzar = fun_cruzar
        self.fun_mutar = fun_mutar
        self.fun_fitness = fun_fitness
        self.longitud_individuo = longitud_individuo
    
    def decodificar(self, genotipo):
        #Devuelve el fenotipo a partir del genotipo
        fenotipo = self.fun_decodificar(genotipo)
        return fenotipo
    
    def cruzar(self, cromosoma1, cromosoma2):         
        #Devuelve el cruce de un par de cromosomas
        cruce = self.fun_cruzar(cromosoma1, cromosoma2)
        return cruce 
    
    def mutar(self, cromosoma, prob):
        #Devuelve el cromosoma mutado
        mutante = self.fun_mutar(cromosoma, prob)
        return mutante
    
    # Si se quisiera implementar otro mecanismo de cruza
    #def cruza_loca(self, cromosoma1, cromosoma2, cromosoma3, cromosoma4):
    #    cruce = self.fun_cruza(cromosoma1, cromosoma2, cromosoma3, cromosoma4)
    #    return cruce

    def fitness(self, cromosoma):
        #Función de valoración
        valoracion = self.fun_fitness(cromosoma)
        return valoracion

In [93]:
def fun_decodificar(genotipo):
    return [binario_a_decimal(genotipo[:4]), binario_a_decimal(genotipo[4:])] 

def fun_cruzar(cromosoma1, cromosoma2):
    # Cruza los cromosomas por la mitad
    l = len(cromosoma1)
    cruce1 = cromosoma1[0:int(l / 2)] + cromosoma2[int(l / 2):l]
    cruce2 = cromosoma2[0:int(l / 2)] + cromosoma1[int(l / 2):l]
#     cruce1 = cromosoma1[0:int(l / 4)] + cromosoma2[int(l / 4):int(l/2)] +
#     cromosoma1[int(l / 2)0:int(l / 4) + int (l / 2)] + cromosoma2[int(l / 4) + int (l / 2):l]
#     cruce2 = cromosoma2[0:int(l / 4)] + cromosoma1[int(l / 4):int(l/2)] +
#     cromosoma2[int(l / 2)0:int(l / 4) + int (l / 2)] + cromosoma1[int(l / 4) + int (l / 2):l]
    return [cruce1, cruce2]

def fun_mutar(cromosoma, prob):
    # Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob
    l = len(cromosoma)
    p = random.randint(0, l - 1)
    if prob > random.uniform(0, 1):
        cromosoma[p] = (cromosoma[p] + 1) % 2
        #cromosoma[p] = cromosoma[p]*-1
    return cromosoma

In [94]:
def poblacion_inicial(problema_genetico, tamano_poblacion):
    l = []
    for i in range(tamano_poblacion):
        l.append([random.choice(problema_genetico.genes) for g in range(problema_genetico.longitud_individuo)])                
    return l

In [95]:
def cruza_padres(problema_genetico, padres):
    l = []
    l1 = len(padres)
#     while len(padres) > 0:
    while padres != []:
        l.extend(problema_genetico.cruzar(padres[0], padres[1]))
        padres.pop(0)
        padres.pop(0)
    return l

In [96]:
def muta_individuos(problema_genetico, poblacion, prob):
    return [problema_genetico.mutar(individuo, prob) for individuo in poblacion]

In [97]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    # Selección por torneo de n individuos de una población. 
    # k es el nº de participantes
    # y opt la función max o min.
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion, k)
        seleccionado = opt(participantes, key = problema_genetico.fitness)
        #opt(poblacion, key = problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

In [98]:
def nueva_generacion_t(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k, opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2 + cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

In [99]:
def algoritmo_genetico_t(problema_genetico, k, opt, ngen, size, prop_cruces, prob_mutar):
    poblacion = poblacion_inicial(problema_genetico, size)
    print("Poblacion Inicial")
    print(poblacion)
    n_padres = round(size * prop_cruces)
    n_padres = int (n_padres if n_padres % 2 == 0 else n_padres - 1)
    n_directos = size - n_padres
    for _ in range(ngen):
        poblacion = nueva_generacion_t(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar)
        print("Nueva población")
        print(poblacion)
    mejor_cr = opt(poblacion, key = problema_genetico.fitness)
    mejor = problema_genetico.decodificar(mejor_cr)
    return (mejor, problema_genetico.fitness(mejor_cr)) 


In [100]:
def decodifica_mochila(cromosoma, n_objetos, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n_objetos):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i] == 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l 

In [101]:
def fitness_mochila(cromosoma, n_objetos, pesos, capacidad, valores):
    objetos_en_mochila = decodifica_mochila(cromosoma, n_objetos, pesos, capacidad)
    valor = 0
    for i in range(n_objetos):
        if objetos_en_mochila[i] == 1:
            valor += valores[i]
    return valor

In [102]:
# _______________________________________________________
# Problema de la mochila 1:
# 10 objetos, peso máximo 165
pesos1 = [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]
valores1 = [1, 2, 2, 1, 0, 3, 5, 4, 8, 7]
# Solución óptima= [1,1,1,1,0,1,0,0,0,0], con valor 309
# _______________________________________________________
# Problema de la mochila 2:
# 15 objetos, peso máximo 750

pesos2 = [70, 73, 77, 80, 82, 87, 90,94,98,106,110,113,115,118,120]
valores2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

## Solución óptima= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1] con valor 1458
## _______________________________________________________
## Problema de la mochila 3:
## 24 objetos, peso máximo 6404180
pesos3 = [382745,799601,909247,729069,467902, 44328,
      34610,698150,823460,903959,853665,551830,610856,
      670702,488960,951111,323046,446298,931161, 31385, 496951,264724,224916,169684]
valores3 = [825594,1677009,1676628,1523970, 943972,  97426,
      69666,1296457,1679693,1902996,
      1844992,1049289,1252836,1319836, 953277,2067538, 675367,
      853655,1826027, 65731, 901489, 577243, 466257, 369261]

# Solución óptima= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1] con valoración 13549094
# _______________________________________________________


In [103]:
def fitness_mochila_1(cromosoma):
    v = fitness_mochila(cromosoma, 10, pesos1, 165, valores1)
    return v

def decodifica_mochila_1(cromosoma):
    v = decodifica_mochila(cromosoma, 10, pesos1, 165)
    return v

def fitness_mochila_2(cromosoma):
    v = fitness_mochila(cromosoma, 15, pesos2, 150, valores2)
    return v

def decodifica_mochila_2(cromosoma):
    v = decodifica_mochila(cromosoma, 15, pesos2, 750)
    return v

m1g = Problema_Genetico([0,1], decodifica_mochila_1, fun_cruzar, fun_mutar, fitness_mochila_1, 10)
m2g = Problema_Genetico([0,1], decodifica_mochila_2, fun_cruzar, fun_mutar, fitness_mochila_2, 15)


In [104]:
print(algoritmo_genetico_t(m2g, 3, max, 100, 50, 0.8, 0.05))

Poblacion Inicial
[[0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1], [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0], [1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0], [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1], [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0